In [ ]:
!pip install transformers -q

     |████████████████████████████████| 2.6 MB 29.3 MB/s 
     |████████████████████████████████| 636 kB 48.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.3 MB/s 
     |████████████████████████████████| 895 kB 53.1 MB/s 


In [ ]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

import tensorflow as tf

from pathlib import Path
from sklearn.model_selection import train_test_split

df = pd.read_csv('augmented_commands.csv')
df

,Question,BaseCommand,CommandID,CommandType
0,how do i access email on my phone,orbit,0,none
1,how do i share documents with colleagues,orbit,0,none
2,access work from mobile device,orbit,0,none
3,how do i get orbit suite,orbit,0,none
4,android work applications,orbit,0,none
...,...,...,...,...
315,how can i speak to renest,call,1,name
316,speak to call ista,call,1,name
317,speak to forebode ista,call,1,name
318,how can i phone ernest,call,1,name


In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=0, stratify=df['BaseCommand'])

In [ ]:
train_texts = train.to_numpy()[:, 0].tolist()
train_labels = train.to_numpy()[:, 2].tolist()
test_texts = test.to_numpy()[:, 0].tolist()
test_labels = test.to_numpy()[:, 2].tolist()

In [ ]:
print(len(train_texts))
print(len(test_texts))

256
64


In [ ]:
print(test_texts)
print(test_labels)

['what is the kerberos of c allista', 'how can i quest a vendor review', 'how manage i append bengaluru time on my world clock', 'where do i request a control override', 'iphone work applicaitons', 'access work from mobile device', 'where to get selfservice tech support', 'how do i access email on my phone', 'how can i request a vendor re view', 'report a security incident', 'how to add wordart no powerpoint', 'how to change page oriehtation in microsoft word', 'how to launch helpdesk', 'send message to binitha', 'how can i ring ernest', 'where can i learn about common architecture patterns', 'where can i preview development products', 'how do i remove time panel', 'how do i request a site unblock', 'how to skype binitha', 'how serve i access emai l on my phone', 'how can i send skype message to peng fei', 'where do i write a new email', 'skype ernest', 'require ass istance on rebooting nds', 'how can i telephone ernest', 'how coiffure i send an email to khooer', 'how do i launch lumos

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=20,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    weight_decay=0.01,    
    eval_steps=10,
    evaluation_strategy='steps',          
)

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=10)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_199', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset             
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:114: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/master/examples/tensorflow
  FutureWarning,


In [ ]:
values = df[['BaseCommand', 'CommandID']].value_counts().keys().tolist()
mapper = {k: v for v, k in values}
print(mapper)

In [ ]:
#@title test your predictions here! \o/

text = 'send a mail to' #@param {type:"string"}

input_encodings = tokenizer([text], truncation=True, padding=True)
input_dataset = tf.data.Dataset.from_tensor_slices((dict(input_encodings), [123456789])) # label doesn't matter since we're testing lol
input_predictions = trainer.predict(input_dataset)
print(input_predictions)
print()
print("Predicted Command:", mapper[np.argmax(input_predictions.predictions)])

In [ ]:
# save the model into a folder called 'checkpoint'
trainer.save_model('checkpoint')